In [3]:
from cassiopeia.TreeSolver import utilities

import pandas as pd 
import numpy as np
import networkx as nx
import pickle as pic

import sys

### Preprocess

https://github.com/YosefLab/Cassiopeia/blob/master/notebooks/reconstruct_lineages.ipynb

In [5]:
alleletable_fp = "GSE146712_IVLT-2B_ALL.alleleTable.txt"
target_lg = 4
alleletable = pd.read_csv(alleletable_fp, sep='\t')


In [6]:
allele_props = utilities.get_indel_props(alleletable, group_var=['intBC', 'lineageGrp'])

Counting unique alleles: 100%|██████████| 117/117 [00:00<00:00, 1328.79it/s]


In [7]:
lg = alleletable[alleletable["lineageGrp"] == target_lg]
cm_fp = "lg4_character_matrix.txt" # relative file path to the cm

utilities.alleletable_to_character_matrix(lg, cm_fp, mutation_map=allele_props, write=True)


Processing characters:  56%|█████▌    | 15/27 [00:00<00:00, 140.53it/s]

Dropping the following intBCs due to lack of diversity with threshold 1.0: []


Processing characters: 100%|██████████| 27/27 [00:00<00:00, 116.43it/s]


### Run reconstruct-lineage Command
reconstruct-lineage --greedy --verbose lg4_character_matrix.txt output.pkl

### Post-Process

In [8]:
g = nx.read_gpickle("output.pkl")
cm = pd.read_csv("lg4_character_matrix.txt", sep='\t', index_col = 0) # read in the associated character matrix

In [9]:
g = g.post_process(cm = cm) # Cassiopeia-Tree's have built in functionality to post-process, callable as a method on the object 
# create a newick string for the given post-processed tree. Note that you need to pass in the 
# networkx object for this

g.newick = utilities.convert_network_to_newick_format(g.network) 

In [13]:

with open('test.nwk', 'w') as f:
    f.write(g.newick)

In [14]:
## now save final tree
out_fp = "test_lg4_tree.processed.pkl"

# pic.dump(g, open(out_fp, 'wb'))

with open('test_lg4_tree.processed.nwk', 'w') as f:
    f.write(out_fp)